In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv('data/train.csv', index_col='Id')
test_data = pd.read_csv('data/test.csv', index_col='Id')

train_data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [5]:
train_data.drop(train_data[(train_data['GrLivArea'] > 400) & (train_data['SalePrice'] < 300000)].index, inplace=True)

In [6]:
train_data[train_data['PoolQC'].notna()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1183,60,RL,160.0,15623,Pave,NaN,IR1,Lvl,AllPub,Corner,...,555,Ex,MnPrv,NaN,0,7,2007,WD,Abnorml,745000


In [12]:
import aqueduct as aq

In [ ]:
@aq.op
def fill_none(raw_house_data):
    none_cols = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual'
                 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', "MasVnrType",
                 'MSSubClass']
    for col in none_cols:
        raw_house_data[col].fillna('None', inplace=True)
        
    return raw_house_data

In [15]:
@aq.op
def fill_zero(none_filled):
    zero_cols = ['GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                 'BsmtFullBath', 'BasmtHalfBath', 'MasVnrArea']
    for col in zero_cols:
        none_filled[col].fillna(0, inplace=True)
        
    return none_filled

In [16]:
@aq.op
def fill_modal_values(na_filled):
    modal_cols = ['MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']
    
    for col in modal_cols:
        na_filled[col].fillna(na_filled[col].mode()[0], inplace=True)
        
    return na_filled

In [17]:
@aq.op
def misc_data_cleaning(filled_data):
    filled_data.drop(['Utilities'], axis=1, inplace=True)
    filled_data['Functional'].fillna('Typ', inplace=True)
    
    filled_data['LotFrontage'] = \
        filled_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillname(x.median()))
    
    return filled_data

In [9]:
@aq.op
def encode_labels(cleaned_data): 
    cleaned_data['MSSubClass'].astype(str, inplace=True)
    cleaned_data['OverallCond'].astype(str, inplace=True)
    cleaned_data['YrSold'].astype(str, inplace=True)
    cleaned_data['MoSold'].astype(str, inplace=True)
    
    from sklearn.preprocessing import LabelEncoder
    categorical_cols = [
        'FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold'
    ]
    
    for col in categorical_cols: 
        enc = LabelEncoder()
        enc.fit(list(claned_data[col].values))
        cleaned_data[col] = enc.transform(list(cleaned_data[col].values))
    
    return cleaned_data

In [10]:
@ap.op
def add_total_sq_ft(encoded): 
    encoded['TotalSF'] = encoded['TotalBsmtSF'] + encoded['1stFlrSF'] + all_data['2ndFlrSF']

In [11]:
@aq.op
def unskew_features(all_features)